# problem1

In [21]:
# prepress the data 
!python preprocess.py train.trees > train.trees.pre
#run train.trees.pre through unknown.py save the output to train.trees.pre.unk
!python unknown.py train.trees.pre > train.trees.pre.unk

In [22]:
#learn a probabilistic CFG from trees

In [23]:
!python new_tree.py < train.trees.pre.unk > grammer.txt

In [20]:
#solve the grammer remove /n and blank line
f = open("grammer.txt",'r')
g = open("new_grammer.txt",'w')
try:   
    while True:        
        line=f.readline()        
        if len(line)==0:           
            break        
        if line.count('\n')==len(line):            
            continue        
        g.write(line)
finally:    
    f.close()    
    g.close()

In [19]:
# use a dict key is every grammer and value is responding counter name 
#sort the dict and we could get the most popular grammer
grammer_dict = {}
file = open("new_grammer.txt")
for line in file.readlines():
    line=line.strip('\n')
    if len(line.split())==3:
        line = ' '.join(line.split()[:2])+' '+line.split()[2].lower()
    if(grammer_dict.has_key(line)):
        grammer_dict[line] = grammer_dict.get(line)+1
    else:
        grammer_dict[line] = 1
sorted(grammer_dict.items(),key = lambda x:x[1])
index = max(grammer_dict,key=grammer_dict.get)
for v,k in grammer_dict.items():
    print v,k

NNP -> salt 6
NP_NNP -> tacoma 10
NP_PRP -> me 82
VB -> leave 4
SQ* -> NP PP 1
SQ* -> NP NP 2
FRAG_WHNP -> WHNP* PP 32
NN -> stop 7
NNP -> southwest 4
TOP -> FRAG_NP_NNP PUNC 1
S_VP -> VBP PP 2
NP -> NP* NNS 34
NP -> NP* NNP 30
NNP -> u 4
NNP -> t 4
NNP -> w 6
WHNP -> WRB RB 5
NNP -> p 1
NNP -> s 3
NP -> NP ADJP_JJ 1
SYM -> <unk> 3
S_VP -> VP* INTJ_UH 2
NNP -> k 4
NNP -> j 4
NP -> NN NN 9
NNP -> d 5
NNP -> f 6
NNP -> a 10
VP -> VBG NP 6
NP -> NP* CD 17
NP -> JJS NN 8
S_VP -> VP* NP 73
NN -> airfare 5
QP* -> IN JJS 2
PP_IN -> from 1
POS -> 's 2
NN -> flight 78
VBN -> <unk> 1
SQ -> VBZ NP 22
NP_NNP -> atlanta 6
PP -> TO NP 58
FRAG_NP -> NP_NNS PP 7
INTJ_UH -> oh 1
NNP -> las 24
VP* -> VP CC 7
NP* -> NP NN 1
FRAG_NP_NN -> <unk> 2
NP* -> NP NP 2
VB -> fly 5
SQ_VP -> VBZ NP 1
NP -> JJ NN 11
NNP -> county 4
ADVP -> RB PP 1
VBZ -> is 38
NNP -> international 2
FRAG* -> WHNP PP 2
VB -> serve 7
DT -> the 128
S_VP -> VP* PP 4
S_VP -> VBP ADVP 1
VBG -> arriving 12
SQ* -> NP_NP_EX SQ* 17
NP* -> NN 

There are 752 rules in my grammer ,the most frequent rule is PUNC -> . it occurs 346 times

# problem 2

In [24]:
import copy
from collections import defaultdict
# transfer the rules with frequency into the probability
count_sum = 0
for key in grammer_dict.keys():
    count_sum = count_sum + grammer_dict.get(key)
#count_sum is the total number of the rules
grammer_freq=defaultdict(int)
for key,value in grammer_dict.items():
    k = key.split(' -> ')[0]
    grammer_freq[k]+=value
grammer_final={}
for key,value in grammer_dict.items():
    k = key.split(' -> ')[0]
    grammer_final[key]=value*1.0/grammer_freq[k] 

In [25]:
#transfer the sentence into seperate word

def to_word_list(sentence):
    word_list = sentence.split(" ")
    return word_list

In [38]:
#f_string is the first line of dev.strings, we transfer it into input_string and delete \n at the end
#of sentence and pass it into the to_word_list transfer it into a word list
f_string = open('dev.strings')
input_string = f_string.readlines()[0].strip('\n')
l =  to_word_list(input_string)
l0 = []
for word in l:
    l0.append(word.lower())
print l
print l0 


['The', 'flight', 'should', 'be', 'eleven', 'a.m', 'tomorrow', '.']
['the', 'flight', 'should', 'be', 'eleven', 'a.m', 'tomorrow', '.']


In [27]:
#grammer_prob is a dict whose key are rules and values are corresponding probability\
#word_search_children is a dict whose key is string after -> value is rule
#word_search_parent is a dict whose key is rule -> value is string before ->
#grammer_prob is a dict whose key is rule -> value is probability
from math import log
from collections import defaultdict
import re
#grammer_prob is a grammer with probablity
grammer_prob = copy.deepcopy(grammer_dict)

for key in grammer_dict.keys():
    grammer_prob[key] = log(grammer_final[key],10)

word_search_children = defaultdict(list)
for k, v in grammer_prob.items():
    word_search_children[k.split("->")[-1].strip()].append(k)


word_search_parent = defaultdict(list) 
for k,v in grammer_prob.items():
    word_search_parent[k].append(k.split("->")[0].strip())


In [40]:
#write a parsing and take the grammer and senctence as input, and output the highest_probability parse
# import copy
def initial_table(sentence,children_list,rule_prob):
    l0 = []
    l = to_word_list(sentence)
    for word in l:
        l0.append(word.lower())
    length = len(l0)
    table = [[[] for i in range(length)] for i in range(length)] 
    for i in range(len(l0)):
        if(l0[i] in children_list.keys()):
            grammer_key = children_list[l0[i]]
            for word in grammer_key:
                table[i][i].append( (word_search_parent[word][0],rule_prob[word]) )
        else:
            grammer_key = children_list["<unk>"]
            for word in grammer_key:
                table[i][i].append( (word_search_parent[word][0],rule_prob[word]) )
#     table_dict = {}
#     table_dict["table"] = table
#     table_dict["table_prob"] = table_prob
    return table
result = initial_table(input_string,word_search_children,grammer_prob) 
for line in result:
    print line

#use initial_table we could get a table on the diagnol with the word of length1 and its label       

[[('DT', -0.18945522061366268)], [], [], [], [], [], [], []]
[[], [('NN', -0.5566117172145995), ('NP_NN', -0.8624721360836793)], [], [], [], [], [], []]
[[], [], [('MD', -0.8129133566428555)], [], [], [], [], []]
[[], [], [], [('VB', -1.919078092376074)], [], [], [], []]
[[], [], [], [], [('CD', -1.845098040014257)], [], [], []]
[[], [], [], [], [], [('RB', -0.8211858826088454)], [], []]
[[], [], [], [], [], [], [('NP_NN', -0.9294189257142926), ('NN', -1.9715850651854174)], []]
[[], [], [], [], [], [], [], [('PUNC', -0.1320967439223066)]]


In [41]:
#use a funciton index_table it add a column to the original table in order to change index of it
def index_table(table):
    for l in table:
        l.insert(0,[])
    return table

for line in index_table(result):
    print line

[[], [('DT', -0.18945522061366268)], [], [], [], [], [], [], []]
[[], [], [('NN', -0.5566117172145995), ('NP_NN', -0.8624721360836793)], [], [], [], [], [], []]
[[], [], [], [('MD', -0.8129133566428555)], [], [], [], [], []]
[[], [], [], [], [('VB', -1.919078092376074)], [], [], [], []]
[[], [], [], [], [], [('CD', -1.845098040014257)], [], [], []]
[[], [], [], [], [], [], [('RB', -0.8211858826088454)], [], []]
[[], [], [], [], [], [], [], [('NP_NN', -0.9294189257142926), ('NN', -1.9715850651854174)], []]
[[], [], [], [], [], [], [], [], [('PUNC', -0.1320967439223066)]]


In [46]:
#after all the initialization process we could implement the parse of a sentence
from math import log

def argmax_top(table):
    top =  table[0][-1]
    if top == []:
        return []
    if len(top) ==1:
        return top[0]
    max = 0
    for i in range(1,len(top)):
        if(top[i][1]>top[max][1]):
            max = i
    return top[max]

def parsing_table(sentence,children_list,grammer_prob):
    table_with_prob = initial_table(sentence,children_list,grammer_prob)
    cky_table = index_table(table_with_prob)
    n = len(cky_table[0])
    for width in range(2,n):
        for start in range(0,n-width):
            end = start+width
            for mid in range(start+1,end):
                list_y = cky_table[start][mid]
                list_z = cky_table[mid][end]
                index_w = 0
                for index_y,y in enumerate(list_y):
                    for index_z,z in enumerate(list_z):
                        tag_y = y[0]
                        prob_y = y[1]
                        tag_z = z[0]
                        prob_z = z[1]
                        tag_x = tag_y + " " + tag_z
                        if(tag_x in children_list):
                            x_yz = children_list[tag_x]
                            for w in x_yz:
                                new_tag = word_search_parent[w][0]
                                prob_x = prob_y + prob_z + grammer_prob[w]
                                cky_table[start][end].append((new_tag,prob_x,start,mid,end,index_y,index_z,index_w))
                                index_w = index_w + 1
# index_z is the index of element in table[mid][end] table[mid][end] is a list 
# index_y is the index of element in table[start][mid],table [mid][end] is a list                          
    return cky_table
                                
table1 = parsing_table(input_string,word_search_children,grammer_prob)
best_top = argmax_top(table1)
print best_top



('TOP', -16.223114671461772, 0, 7, 8, 13, 0, 7)


In [45]:
#use backtrack method to get route with the highest probability
def backtrack(cky_table,i,j,k,sentence):
    if(j == i+1):
        return "("+cky_table[i][j][k][0] + " "+ sentence[i]+")"
    item = cky_table[i][j][k]
    new_tag=item[0]
    start1,end1 = item[2:4]
    start2,end2 = item[3:5]
    k1 = item[5]
    k2 = item[6]
    return '('+new_tag +' '+backtrack(cky_table,start1,end1,k1,sentence)+' '+backtrack(cky_table,start2,end2,k2,sentence)+')'   
q2_result = backtrack(table1,best_top[2],best_top[4],best_top[-1],l)
print q2_result

(TOP (S (NP (DT The) (NN flight)) (VP (MD should) (VP (VB be) (NP (NP* (CD eleven) (RB a.m)) (NN tomorrow))))) (PUNC .))


Problem2 answer : the output of parser on the first line of dev.strings is above and 
the corresponding log probability is -17.745297989080456

In [47]:
f_test = open('dev.strings')
input_test = f_test.readlines()[0].strip('\n')
print input_test
table_test = parsing_table(input_test,word_search_children,grammer_prob)
print argmax_top(table_test)


The flight should be eleven a.m tomorrow .
('TOP', -16.223114671461772, 0, 7, 8, 13, 0, 7)


In [ ]:
# problem 3

In [48]:
from math import log
import time

sentence_length = []
parsing_time = []
file_p3 = open("dev.strings",'r')
for line in file_p3:
    line = line.strip('\n')
    word_list_q3 = to_word_list(line)
    length_q3 = len(word_list_q3)
    start_time = time.clock()
    table_q3 = parsing_table(line,word_search_children,grammer_prob)
    best_top_q3 = argmax_top(table_q3)
    if(not best_top_q3 == []):
        q3_result  = backtrack(table_q3,best_top_q3[2],best_top_q3[4],best_top_q3[-1],word_list_q3)
    else:
        continue
    interval =time.clock() - start_time
    sentence_length.append(log(length_q3,10))
    parsing_time.append(log(interval,10))
print len(sentence_length)

53


In [49]:
#using sentence length x axis using parsing time as y axis 
#make a plot 
import matplotlib.pyplot as plt
import numpy as np


x_length_q3 = np.array(sentence_length)
y_time_q3 = np.array(parsing_time)

f1 = np.polyfit(x_length_q3,y_time_q3,1)
y_values = np.polyval(f1,x_length_q3)
print f1

plt.figure
plt.xlabel('x')
plt.ylabel('y')
plt.scatter(sentence_length,parsing_time,label = "scatter plot")
plt.plot(x_length_q3,y_values,label = "ployfit curve")
plt.show()

[ 3.09909634 -5.76856963]


<Figure size 640x480 with 1 Axes>

Answer to Q3: we plot the scatter plot of parsing time(y axis) versus sentence length(x axis),and we ployfit the scatter plot for a functional curve with log-scale. the first parameter is nearly 3 it seems a curve y = x^3 transfered into a log-form for both side , the reason is that when we implement the parsing who has 3 for loops
( width, start, mid ) ,so the time complexity is O(n^3)

# problem 4

In [50]:
# transfer the answer of parser output to the q4_result.txt
with open("dev.strings",'r') as f_q4_read:
    with open("q4_result_test.txt",'w') as f_q4_write:
        for line in f_q4_read:
            line = line.strip('\n')
            word_list_q4 = to_word_list(line)
            table_q4 = parsing_table(line,word_search_children,grammer_prob)
            best_top_q4 = argmax_top(table_q4)
            if(not best_top_q4 == []):
                q4_result  = backtrack(table_q4,best_top_q4[2],best_top_q4[4],best_top_q4[-1],word_list_q4)
            else:
                q4_result  = ""
            f_q4_write.write(q4_result+'\n')


In [51]:
!python postprocess.py q4_result_test.txt > q4_result_test_final.txt

In [52]:
!python evalb.py q4_result_test_final.txt dev.trees

q4_result_test_final.txt	440 brackets
dev.trees	474 brackets
matching	405 brackets
precision	0.920454545455
recall	0.854430379747
F1	0.886214442013


Problem5 method3: Change the input word into lowercase, For example in the grammer word ,all and All should be treated as one word, So I change the input sentence and the word in grammar into lowercase,then apply the pcfg to it the f1 score has been improved to 0.886


# problem 5

In [53]:
# first improvememt is using smoothing method to 
# for v,k in grammer_dict.items():
#     print v,k
import copy
from copy import deepcopy
from collections import defaultdict

new_dict_freq = copy.deepcopy(grammer_dict)
for key in new_dict_freq.keys():
    new_dict_freq[key] = new_dict_freq[key]+0.001
    
grammer_freq_q5=defaultdict(int)
for key,value in new_dict_freq.items():
    k = key.split(' -> ')[0]
    grammer_freq_q5[k]+=value
    
grammer_final_q5={}
for key,value in new_dict_freq.items():
    k = key.split(' -> ')[0]
    grammer_final_q5[key]=value*1.0/grammer_freq_q5[k] 
# for v, k in grammer_final.items():
#     print v,k
# for v, k in grammer_final_q5:
#     print v,k

grammer_prob_q5 = copy.deepcopy(grammer_dict)

for key in grammer_dict.keys():
    grammer_prob_q5[key] = log(grammer_final_q5[key],10)

print grammer_prob["NP_PRP -> me"]
print grammer_prob_q5["NP_PRP -> me"]

-0.219725055987
-0.219725162691


In [51]:
with open("dev.strings",'r') as f_q4_read:
    with open("q5_result.txt",'w') as f_q4_write:
        for line in f_q4_read:
            line = line.strip('\n')
            word_list_q4 = to_word_list(line)
            table_q4 = parsing_table(line,word_search_children,grammer_prob_q5)
            best_top_q4 = argmax_top(table_q4)
            if(not best_top_q4 == []):
                q4_result  = backtrack(table_q4,best_top_q4[2],best_top_q4[4],best_top_q4[-1],word_list_q4)
            else:
                q4_result  = ""
            f_q4_write.write(q4_result+'\n')

In [52]:
!python postprocess.py q5_result.txt > q5_improvement1
!python evalb.py q5_improvement1 dev.trees

q5_improvement1	436 brackets
dev.trees	474 brackets
matching	400 brackets
precision	0.917431192661
recall	0.84388185654
F1	0.879120879121


Problem5: method 1: Using Add-One smoothing for the training data, I add 1 to all counts and recompute the probabilty, However it did not help for improving my parser, the reason is that add 1 smoothing method smooths too much 

In [54]:
with open("dev2.strings",'r') as f_q4_read:
    with open("q5_result2.txt",'w') as f_q4_write:
        for line in f_q4_read:
            line = line.strip('\n')
            word_list_q4 = to_word_list(line)
            table_q4 = parsing_table(line,word_search_children,grammer_prob_q5)
            best_top_q4 = argmax_top(table_q4)
            if(not best_top_q4 == []):
                q4_result  = backtrack(table_q4,best_top_q4[2],best_top_q4[4],best_top_q4[-1],word_list_q4)
            else:
                q4_result  = ""
            f_q4_write.write(q4_result+'\n')

In [53]:
!python postprocess.py q5_result2.txt > q5_improvement2
!python evalb.py q5_improvement2 dev2.trees

q5_improvement2	187 brackets
dev2.trees	264 brackets
matching	18 brackets
precision	0.096256684492
recall	0.0681818181818
F1	0.079822616408


Problem5 method2, As I used add-one smoothing method,but it seemd smoothing too much for the given data. So as a result, I consider using add-lambda smoothing(lambda we choose here as 0.001).and result are above.However it have no help for improving the F1 score

In [56]:
!python rbranch.py -i dev.strings -g new_grammer.txt -o test_rbranch.txt
!python postprocess.py test_rbranch.txt > dev.parses.post
!python evalb.py dev.parses.post dev.trees

dev.parses.post	435 brackets
dev.trees	474 brackets
matching	400 brackets
precision	0.919540229885
recall	0.84388185654
F1	0.880088008801


In [25]:
!python preprocess.py < train.trees | python unknown.py > tmpfile